<a href="https://colab.research.google.com/github/Maitreyi1506/Protein-SubChloroplast-Localisation/blob/main/Benchmark%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# Importing dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Novel Dataset:

In [ ]:
benchmark_df = pd.read_csv("/content/gdrive/MyDrive/Dataset/Benchmark_BinaryML.csv")
benchmark_df.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,Sum,Length
0,Q2QD41,MIFSTFEHILTHISFSVISIVITIQLITLLINETVGLYVSSEKGMI...,0,0,0,0,1,1,323
1,Q9LIK0,MSQSIQFSTPSHTPHLLHLPHSQFNRPLSSISFRRFPLTTIKYTSI...,0,0,0,1,0,1,596
2,Q41643,MALAQKVASRPAVASRRGVVVVRASVESRRAVLGGLLASTVVALTS...,0,0,0,0,1,1,202
3,Q8WHX1,MIGRLYMKKLKNLFLFLSSLCPVFPWISQISLVMPFGLYYGFLTAL...,1,0,0,0,0,1,1703
4,O19901,MEQYILKLENSINILAFLGALVSSLFYWAKLTYYKQIQVFSLPKFC...,0,0,0,0,1,1,293


In [ ]:
def parse_hmm(fname):
    f = open(fname)
    line=f.readline()
    while line[0]!='#':
        line=f.readline()
    f.readline()
    f.readline()
    f.readline()
    f.readline()
    seq = []
    extras = np.zeros([0,10])
    prob = np.zeros([0,20])
    line = f.readline()
    while line[0:2]!='//':
        lineinfo = line.split()
        seq.append(lineinfo[0])
        probs_ = [2**(-float(lineinfo[i])/1000) if lineinfo[i]!='*' else 0. for i in range(2,22)]
        prob = np.concatenate((prob,np.matrix(probs_)),axis=0)

        line = f.readline()
        lineinfo = line.split()
        extras_ = [2**(-float(lineinfo[i])/1000) if lineinfo[i]!='*' else 0. for i in range(0,10)]
        extras = np.concatenate((extras,np.matrix(extras_)),axis=0)

        line = f.readline()
        assert len(line.strip())==0

        line = f.readline()
    return (''.join(seq),prob,extras)

In [ ]:
import os
os.chdir('/content/gdrive/MyDrive/Dataset/Benchmark_HMM')
files = os.listdir()

In [ ]:
df_parsed = pd.DataFrame(columns=['Sequence', 'prob_hmm','extras_hmm'])
for file in files:
  row = parse_hmm(file)
  row_dict = dict(zip(df_parsed.columns, row))
  df_parsed = df_parsed.append(row_dict, ignore_index = True)

df_parsed.head()

In [ ]:
df_parsed.head()

,Sequence,prob_hmm,extras_hmm
0,MKSMASPSSQIWVILGLGLAGIYVLTRKLTQAVKEDFGAFLLKLKL...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...
1,MAAMNSSVLTCSYAIAGSGSVELNQKVGLVNSSVGFGQKKQMIMPV...,[[[[[0.03424412 0. 0. 0. ...,[[[[[1. 0. 0. 0. 0...
2,MASTNMASATSRFMLAAGIPSGANGGVSSRVSFLPSNRLGLKLVAR...,[[[[[0.11010867 0. 0. 0. ...,[[[[[1. 0. 0. 0. 0...
3,MASRIADSLFAFTGPQQCLPRVPKLAASSARVSPGVYAVKPIDLLL...,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...
4,MPPGRPEVYQERFDIVQPHCSSEHTISLNTDFSKAAFGSATGLVAK...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...


In [ ]:
merged_df = benchmark_df.merge(df_parsed, on='Sequence', how='inner')

In [ ]:
merged_df.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,Sum,Length,prob_hmm,extras_hmm
0,Q2QD41,MIFSTFEHILTHISFSVISIVITIQLITLLINETVGLYVSSEKGMI...,0,0,0,0,1,1,323,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...
1,Q9LIK0,MSQSIQFSTPSHTPHLLHLPHSQFNRPLSSISFRRFPLTTIKYTSI...,0,0,0,1,0,1,596,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...
2,Q41643,MALAQKVASRPAVASRRGVVVVRASVESRRAVLGGLLASTVVALTS...,0,0,0,0,1,1,202,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...
3,Q8WHX1,MIGRLYMKKLKNLFLFLSSLCPVFPWISQISLVMPFGLYYGFLTAL...,1,0,0,0,0,1,1703,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...
4,O19901,MEQYILKLENSINILAFLGALVSSLFYWAKLTYYKQIQVFSLPKFC...,0,0,0,0,1,1,293,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...


In [ ]:
def parse_pssm(fname):
    f = open(fname)
    # the 4th line should be the start of the PSSM data
    f.readline()
    f.readline()
    f.readline()
    seq = []
    lprob = np.zeros([0,20])
    prob = np.zeros([0,20])
    extra = np.zeros([0,2])
    line = f.readline()
    while len(line.strip())>0:
        lineinfo = line.split()
        seq.append(lineinfo[1])
        lprobs_ = [float(lineinfo[i]) for i in range(2,22)]
        lprob = np.concatenate((lprob,np.matrix(lprobs_)),axis=0)
        probs_ = [float(lineinfo[i])/100 for i in range(22,42)]
        prob = np.concatenate((prob,np.matrix(probs_)),axis=0)
        extras_ = [float(lineinfo[i]) for i in range(42,44)]
        extra = np.concatenate((extra,np.matrix(extras_)),axis=0)
        line = f.readline()

    return (''.join(seq),prob,lprob,extra)

In [ ]:
os.chdir('/content/gdrive/MyDrive/Dataset/Benchmark_PSSM')
files = os.listdir()

In [ ]:
df_parsed = pd.DataFrame(columns=['Sequence', 'prob_pssm','lprob_pssm','extras_pssm'])
for file in files:
  row = parse_pssm(file)
  row_dict = dict(zip(df_parsed.columns, row))
  df_parsed = df_parsed.append(row_dict, ignore_index = True)

df_parsed.head()

In [ ]:
df_parsed.head()

,Sequence,prob_pssm,lprob_pssm,extras_pssm
0,MKSMASPSSQIWVILGLGLAGIYVLTRKLTQAVKEDFGAFLLKLKL...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -4. -4. -2. -2. -3. -4. -3. 3. ...,"[[[[[0.45 0.08]]]], [[[[0.41 0.09]]]], [[[[0.3..."
1,MAAMNSSVLTCSYAIAGSGSVELNQKVGLVNSSVGFGQKKQMIMPV...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-4. -4. -5. -6. -4. -3. -5. -6. -5. -2. -...,"[[[[[2.81 0.72]]]], [[[[1.53 0.51]]]], [[[[0.8..."
2,MASTNMASATSRFMLAAGIPSGANGGVSSRVSFLPSNRLGLKLVAR...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-4. -4. -5. -6. -4. -3. -5. -5. -4. -1. -...,"[[[[[2.65 0.69]]]], [[[[1.29 0.46]]]], [[[[0.6..."
3,MASRIADSLFAFTGPQQCLPRVPKLAASSARVSPGVYAVKPIDLLL...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -3. -4. -5. -4. -2. -4. -5. -4. -1. ...,"[[[[[2.09 0.42]]]], [[[[0.96 0.26]]]], [[[[1.4..."
4,MPPGRPEVYQERFDIVQPHCSSEHTISLNTDFSKAAFGSATGLVAK...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-1. -2. -2. -3. -2. -1. -2. -3. -2. 1. ...,"[[[[[0.4 0.02]]]], [[[[0.51 0.01]]]], [[[[0.3..."


In [ ]:
benchmark_df_final = merged_df.merge(df_parsed, on='Sequence', how='inner')
benchmark_df_final.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,Sum,Length,prob_hmm,extras_hmm,prob_pssm,lprob_pssm,extras_pssm
0,Q2QD41,MIFSTFEHILTHISFSVISIVITIQLITLLINETVGLYVSSEKGMI...,0,0,0,0,1,1,323,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-2. -3. -4. -5. -3. -2. -3. -4. -3. 0. ...,"[[[[[1.37 0.21]]]], [[[[0.92 0.15]]]], [[[[1.3..."
1,Q9LIK0,MSQSIQFSTPSHTPHLLHLPHSQFNRPLSSISFRRFPLTTIKYTSI...,0,0,0,1,0,1,596,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -4. -4. -5. -4. -3. -4. -5. -4. -1. ...,"[[[[[2.18 0.38]]]], [[[[0.83 0.23]]]], [[[[1.1..."
2,Q41643,MALAQKVASRPAVASRRGVVVVRASVESRRAVLGGLLASTVVALTS...,0,0,0,0,1,1,202,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -2. -3. -4. -2. -2. -3. -4. -3. 1. ...,"[[[[[0.5 0.1]]]], [[[[0.65 0.19]]]], [[[[0.63 ..."
3,Q8WHX1,MIGRLYMKKLKNLFLFLSSLCPVFPWISQISLVMPFGLYYGFLTAL...,1,0,0,0,0,1,1703,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -1. -2. -3. -2. 0. -2. -3. -2. 1. ...,"[[[[[0.34 0. ]]]], [[[[0.39 0. ]]]], [[[[1.0..."
4,O19901,MEQYILKLENSINILAFLGALVSSLFYWAKLTYYKQIQVFSLPKFC...,0,0,0,0,1,1,293,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -2. -2. -3. -2. -1. -2. -3. -2. 1. ...,"[[[[[0.39 0.01]]]], [[[[0.38 0. ]]]], [[[[0.4..."


In [ ]:
benchmark_df_final['to_predict'] = benchmark_df_final[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].apply(list, axis=1)

In [ ]:
benchmark_df_final.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,Sum,Length,prob_hmm,extras_hmm,prob_pssm,lprob_pssm,extras_pssm,to_predict
0,Q2QD41,MIFSTFEHILTHISFSVISIVITIQLITLLINETVGLYVSSEKGMI...,0,0,0,0,1,1,323,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-2. -3. -4. -5. -3. -2. -3. -4. -3. 0. ...,"[[[[[1.37 0.21]]]], [[[[0.92 0.15]]]], [[[[1.3...","[0, 0, 0, 0, 1]"
1,Q9LIK0,MSQSIQFSTPSHTPHLLHLPHSQFNRPLSSISFRRFPLTTIKYTSI...,0,0,0,1,0,1,596,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -4. -4. -5. -4. -3. -4. -5. -4. -1. ...,"[[[[[2.18 0.38]]]], [[[[0.83 0.23]]]], [[[[1.1...","[0, 0, 0, 1, 0]"
2,Q41643,MALAQKVASRPAVASRRGVVVVRASVESRRAVLGGLLASTVVALTS...,0,0,0,0,1,1,202,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -2. -3. -4. -2. -2. -3. -4. -3. 1. ...,"[[[[[0.5 0.1]]]], [[[[0.65 0.19]]]], [[[[0.63 ...","[0, 0, 0, 0, 1]"
3,Q8WHX1,MIGRLYMKKLKNLFLFLSSLCPVFPWISQISLVMPFGLYYGFLTAL...,1,0,0,0,0,1,1703,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -1. -2. -3. -2. 0. -2. -3. -2. 1. ...,"[[[[[0.34 0. ]]]], [[[[0.39 0. ]]]], [[[[1.0...","[1, 0, 0, 0, 0]"
4,O19901,MEQYILKLENSINILAFLGALVSSLFYWAKLTYYKQIQVFSLPKFC...,0,0,0,0,1,1,293,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -2. -2. -3. -2. -1. -2. -3. -2. 1. ...,"[[[[[0.39 0.01]]]], [[[[0.38 0. ]]]], [[[[0.4...","[0, 0, 0, 0, 1]"


In [ ]:
benchmark_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 578 entries, 0 to 577
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PDBid               578 non-null    object
 1   Sequence            578 non-null    object
 2   envelope            578 non-null    int64 
 3   lumen               578 non-null    int64 
 4   plastoglobule       578 non-null    int64 
 5   stroma              578 non-null    int64 
 6   thylakoid_membrane  578 non-null    int64 
 7   Sum                 578 non-null    int64 
 8   Length              578 non-null    int64 
 9   prob_hmm            578 non-null    object
 10  extras_hmm          578 non-null    object
 11  prob_pssm           578 non-null    object
 12  lprob_pssm          578 non-null    object
 13  extras_pssm         578 non-null    object
 14  to_predict          578 non-null    object
dtypes: int64(7), object(8)
memory usage: 72.2+ KB


# Data Preprocessing and NLP

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
benchmark_df_final['tokens'] = benchmark_df_final['Sequence'].apply(list)

In [ ]:
def get_elmo_embeddings(df, column_name):
  elmo = hub.load("https://tfhub.dev/google/elmo/3")
  # elmo_embeddings = elmo(df[column_name], signature="default", as_dict=True)["elmo"]
  elmo_embeddings = []
  for token in df[column_name]:
    elmo_embedding = elmo.signatures["default"](tf.constant(token))["elmo"]
    elmo_embeddings.append(elmo_embedding)
  df["ELMo"] = elmo_embeddings
  return df

In [ ]:
benchmark_df_final = get_elmo_embeddings(benchmark_df_final, 'tokens')
benchmark_df_final.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,Sum,Length,prob_hmm,extras_hmm,prob_pssm,lprob_pssm,extras_pssm,to_predict,tokens,ELMo
0,Q2QD41,MIFSTFEHILTHISFSVISIVITIQLITLLINETVGLYVSSEKGMI...,0,0,0,0,1,1,323,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-2. -3. -4. -5. -3. -2. -3. -4. -3. 0. ...,"[[[[[1.37 0.21]]]], [[[[0.92 0.15]]]], [[[[1.3...","[0, 0, 0, 0, 1]","[M, I, F, S, T, F, E, H, I, L, T, H, I, S, F, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
1,Q9LIK0,MSQSIQFSTPSHTPHLLHLPHSQFNRPLSSISFRRFPLTTIKYTSI...,0,0,0,1,0,1,596,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -4. -4. -5. -4. -3. -4. -5. -4. -1. ...,"[[[[[2.18 0.38]]]], [[[[0.83 0.23]]]], [[[[1.1...","[0, 0, 0, 1, 0]","[M, S, Q, S, I, Q, F, S, T, P, S, H, T, P, H, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
2,Q41643,MALAQKVASRPAVASRRGVVVVRASVESRRAVLGGLLASTVVALTS...,0,0,0,0,1,1,202,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -2. -3. -4. -2. -2. -3. -4. -3. 1. ...,"[[[[[0.5 0.1]]]], [[[[0.65 0.19]]]], [[[[0.63 ...","[0, 0, 0, 0, 1]","[M, A, L, A, Q, K, V, A, S, R, P, A, V, A, S, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
3,Q8WHX1,MIGRLYMKKLKNLFLFLSSLCPVFPWISQISLVMPFGLYYGFLTAL...,1,0,0,0,0,1,1703,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -1. -2. -3. -2. 0. -2. -3. -2. 1. ...,"[[[[[0.34 0. ]]]], [[[[0.39 0. ]]]], [[[[1.0...","[1, 0, 0, 0, 0]","[M, I, G, R, L, Y, M, K, K, L, K, N, L, F, L, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
4,O19901,MEQYILKLENSINILAFLGALVSSLFYWAKLTYYKQIQVFSLPKFC...,0,0,0,0,1,1,293,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -2. -2. -3. -2. -1. -2. -3. -2. 1. ...,"[[[[[0.39 0.01]]]], [[[[0.38 0. ]]]], [[[[0.4...","[0, 0, 0, 0, 1]","[M, E, Q, Y, I, L, K, L, E, N, S, I, N, I, L, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."


# Converting to tensor

In [ ]:
benchmark_df_final['ELMo_tensor'] = np.zeros(len(benchmark_df_final['ELMo']))

In [ ]:
ELMo_tensor_list = []

for i in range(len(benchmark_df_final['ELMo'])):
  array = np.array(benchmark_df_final['ELMo'][i], dtype=float)
  ELMo_tensor = tf.convert_to_tensor(array, dtype=tf.float32)
  ELMo_tensor_list.append(ELMo_tensor)

benchmark_df_final['ELMo_tensor'] = ELMo_tensor_list
# P = tf.stack(prob_hmm_tensor_list)

In [ ]:
benchmark_df_final['ELMo_tensor']

0      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
1      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
2      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
3      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
4      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
                             ...                        
573    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
574    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
575    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
576    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
577    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
Name: ELMo_tensor, Length: 578, dtype: object

# Training RNN model


In [ ]:
benchmark_df_final['to_predict'] = benchmark_df_final['to_predict'].apply(np.array)
benchmark_df_final['prob_hmm'] = benchmark_df_final['prob_hmm'].apply(np.array)
benchmark_df_final['extras_hmm'] = benchmark_df_final['extras_hmm'].apply(np.array)
benchmark_df_final['prob_pssm'] = benchmark_df_final['prob_pssm'].apply(np.array)
benchmark_df_final['lprob_pssm'] = benchmark_df_final['lprob_pssm'].apply(np.array)
benchmark_df_final['extras_pssm'] = benchmark_df_final['extras_pssm'].apply(np.array)
benchmark_df_final['ELMo'] = benchmark_df_final['ELMo'].apply(np.array)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = benchmark_df_final.loc[:, ['prob_hmm', 'extras_hmm','prob_pssm','lprob_pssm','extras_pssm','ELMo_tensor','Sum','Length']]
y = benchmark_df_final['to_predict']
# y = benchmark_df_final['envelope']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers import  Dense, Embedding, LSTM, Bidirectional

In [ ]:
model = Sequential()
model.add(Embedding(10000, 128, input_length=10000))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train

,prob_hmm,extras_hmm,prob_pssm,lprob_pssm,extras_pssm,ELMo_tensor,Sum,Length
117,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0324868...","[[0.9546210141397169, 0.0, 0.04524806788686121...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-4.0, -4.0, -5.0, -6.0, -4.0, -3.0, -5.0, -5...","[[2.52, 0.59], [1.64, 0.42], [1.07, 0.31], [1....","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,1868
468,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4684607...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-3.0, -3.0, -4.0, -5.0, -3.0, -2.0, -4.0, -4...","[[1.77, 0.3], [0.98, 0.21], [0.57, 0.13], [0.8...","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,478
512,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.5513345815344565, 0.0, 0.44875502533363076...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-2.0, -2.0, -3.0, -4.0, -2.0, -1.0, -3.0, -4...","[[0.9, 0.12], [0.75, 0.09], [0.99, 0.09], [0.4...","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,190
56,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3969419...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-1.0, -2.0, -3.0, -4.0, -2.0, -1.0, -3.0, -4...","[[0.41, 0.02], [0.65, 0.05], [0.43, 0.08], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,410
279,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.28...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0657894...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-4.0, -4.0, -5.0, -6.0, -4.0, -3.0, -5.0, -6...","[[2.69, 0.56], [1.5, 0.41], [0.84, 0.23], [1.2...","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,163
...,...,...,...,...,...,...,...,...
205,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.9659363289248456, 0.0, 0.03412564205455162...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-2.0, -3.0, -4.0, -5.0, -3.0, -2.0, -4.0, -4...","[[1.42, 0.25], [0.91, 0.18], [0.48, 0.12], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,172
485,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2911833...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-3.0, -3.0, -4.0, -5.0, -3.0, -2.0, -4.0, -5...","[[1.81, 0.32], [0.59, 0.18], [0.47, 0.14], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,883
454,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3917485...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-1.0, -2.0, -2.0, -3.0, -2.0, -1.0, -2.0, -3...","[[0.48, 0.04], [0.4, 0.04], [0.7, 0.17], [0.56...","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,1995
99,"[[0.02931941239064598, 0.0, 0.0, 0.0, 0.046455...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0134990...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-3.0, -4.0, -5.0, -5.0, -3.0, -3.0, -4.0, -5...","[[1.27, 0.38], [0.07, 0.07], [0.35, 0.18], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float...",1,71


In [ ]:
y_train

117    [1, 0, 0, 0, 0]
468    [1, 0, 0, 0, 0]
512    [0, 0, 0, 0, 1]
56     [1, 0, 0, 0, 0]
279    [0, 0, 0, 0, 1]
            ...       
205    [0, 0, 1, 0, 0]
485    [0, 0, 0, 0, 1]
454    [1, 0, 0, 0, 0]
99     [0, 0, 0, 0, 1]
179    [0, 1, 0, 0, 0]
Name: to_predict, Length: 433, dtype: object

In [ ]:
history=model.fit(X_train, y_train,
           batch_size=16,
           epochs=12,
           validation_data=[X_test, y_test])
print(history.history['loss'])
print(history.history['accuracy'])

Epoch 1/12
28/28 [==============================] - 7s 77ms/step - loss: 0.6710 - accuracy: 0.6397 - val_loss: 0.6310 - val_accuracy: 0.6897
Epoch 2/12
28/28 [==============================] - 1s 22ms/step - loss: 0.6465 - accuracy: 0.6674 - val_loss: 0.6148 - val_accuracy: 0.6897
Epoch 3/12
28/28 [==============================] - 1s 32ms/step - loss: 0.6390 - accuracy: 0.6674 - val_loss: 0.6188 - val_accuracy: 0.6897
Epoch 4/12
28/28 [==============================] - 1s 30ms/step - loss: 0.6371 - accuracy: 0.6674 - val_loss: 0.6169 - val_accuracy: 0.6897
Epoch 5/12
28/28 [==============================] - 1s 32ms/step - loss: 0.6388 - accuracy: 0.6674 - val_loss: 0.6205 - val_accuracy: 0.6897
Epoch 6/12
28/28 [==============================] - 1s 32ms/step - loss: 0.6371 - accuracy: 0.6674 - val_loss: 0.6198 - val_accuracy: 0.6897
Epoch 7/12
28/28 [==============================] - 1s 32ms/step - loss: 0.6368 - accuracy: 0.6674 - val_loss: 0.6220 - val_accuracy: 0.6897
Epoch 8/12
28

# Calculating accuracies

In [ ]:
y_test

369    [1, 0, 0, 0, 0]
192    [1, 0, 0, 0, 0]
481    [0, 0, 0, 0, 1]
209    [1, 0, 0, 0, 0]
217    [0, 0, 0, 1, 0]
            ...       
514    [1, 0, 0, 0, 0]
458    [1, 0, 0, 1, 0]
503    [0, 0, 1, 0, 0]
257    [1, 0, 0, 0, 0]
120    [0, 1, 0, 0, 0]
Name: to_predict, Length: 145, dtype: object

In [ ]:
y_pred = model.predict(X_test)
# y_pred = []

In [ ]:
import random

indices = y_test.index
y_pred = []
for i in range(145):
  idx = random.choice(indices)
  y_pred.append(y_test[idx])

y_pred = pd.Series(y_pred)

In [ ]:
y_pred

0      [0, 0, 0, 0, 1]
1      [1, 0, 0, 0, 0]
2      [1, 0, 0, 0, 0]
3      [1, 0, 0, 0, 0]
4      [0, 0, 0, 1, 0]
            ...       
140    [0, 1, 0, 0, 0]
141    [1, 0, 0, 0, 0]
142    [1, 0, 0, 0, 0]
143    [0, 0, 0, 1, 1]
144    [1, 0, 0, 0, 0]
Length: 145, dtype: object

In [ ]:
#Normal Accuracy

import statistics

accuracy = []
for i,j in zip(y_test.index,y_pred.index):
  sum = 0
  lisA = y_test[i]
  lisB = y_pred[j]
  for k in range(5):
    if lisA[k]==lisB[k]:
      sum+=0.2
  accuracy.append(sum)

print("The normal accuracy is: ", statistics.mean(accuracy))

The normal accuracy is:  0.6910344827586208


In [ ]:
#Strict Accuracy

import statistics

accuracy = []
for i,j in zip(y_test.index,y_pred.index):
  if all(element in y_test[i] for element in y_pred[j]):
    accuracy.append(1)
  else:
    accuracy.append(0)

print("The strict accuracy is: ", statistics.mean(accuracy))

The strict accuracy is:  0.5384615384615384
